# Apache Kafka: The Father of Distributed Messaging Platforms

## History

Kafka is a distributed streaming platform which was written by Franz Kafka who was a famous author in the 19th century. He wrote a book called "The Metamorphosis".
<div style="display: flex; justify-content: space-between;">
<img src="../images/franz-kafka.jpeg" alt="Franz Kafka" style="max-height: 500px;">

<img src="../images/metamorphosis.jpg" alt="Metamorphosis" style="max-height: 500px;">
</div>

### Just kidding!

It was created by Linkedin. They named it after the famous author Franz Kafka as it is a system that optimizes for "writing"

Kafka is an influence of other distributed streaming platforms such as AWS Kinesis and Google Cloud Pub/Sub.


## Kafka Architecture at Coinsquare



<img src="../images/apache-kafka.png" alt="Kafka Architecture" style="height: 85%;">

## Kafka Concepts


- **Topics**: a queue where messages will be published to

- **Partitions**: enables parallelism in Kafka. One topic can have multiple partitions

- **Apache Avro**: Schema Registry. Used to defined a typed-strong message and validate it before sending out. Messages can be serialized in binary

- **Producer**: Publish messages

- **Messages**: Data being published by producer (Duh) which can then be consumed in downstream services. Messages are immutable and can only be changed by publishing a new message.

- **Offset**: Messages are arrived in order and their position is tracked by an offset.

- **Consumer**: End client that consumes messages from a partition

- **Consumer Group**: Kafka assigns the partitions of a topic to the consumers in a group so that each partition is assigned to one consumer in the group. **Two consumers in the same group will not receive the same message**. However, **Two consumers in different groups can receive the same message**. Distribution rules are defined as:
    - If consumers < partitions: Some consumers will handle multiple partitions
    - If consumers = partitions: Each consumer gets one partition
    - If consumers > partitions: Some consumers will be idle
    - When a consumer fails, the partitions assigned to it will be reassigned to another consumer in the same group.



- **KRaft or ZooKeeper**: Manage Cluster (CPU, Memory, Topic, Partition Balancing). ZooKeeper is a dependency for old version of Kafka whereas KRaft is enabled natively in the new version 2.8+

- **Broker**: A Kafka server node that is responsible for multiple partitions of multiple topics. A broker can be elected to be a leader that is responsible for replicating data across partitions

- **Exactly-once Delivery**: Kafka’s Transactional API can ensure each message to be processed exactly once.

- **At-least-once Delivery**: Messages never lost but can be processed more than once. This happens if offsets are failed to committed (due to network, consumer crash)

- **Dead Letter Queue**: Kafka does not provide built-in DLQ like services such as PubSub or SQS. You have to implement a topic and a library to send to dead letter queue.

- **Retry Strategy**: Kafka does not provide built-in retry mechanism per topic. You have to either do that at Consumer level or create a bunch of retry topics with different delays

- **Kafka Connect**: Kafka connector to connect to external systems such as databases, S3, etc.

## Demo

> Broadcast messages to multiple consumers (One publisher multiple consumers)

```
cd samples/broadcast
python producer.py 
# in another terminal
python consumer.py foo
python consumer.py bar
```

> Chat Room

```
cd samples/chat-room
python chat_client.py huy
python chat_client.py karida
python chat_client.py lily
```

## Gotchas and Best Practices

<img src="../images/kafka-best-practices.jpg" alt="Kafka Best Practices" style="height: 65%; margin: 0 auto;">

> Gotchas

- A message usually has maximum size of 1MB. You can bump this number up but in can affect performance or OutOfMemoryErrors. 

- Sending large messages such as PDF should use an interim Storage System and only be sent as reference.

- Don’t publish plain message. Use a schema registry such as Apache Avro

- acks: set to 0/1/all to ensure durability. With 0, the producer doesn’t wait for acknowledgement. With 1, the producer waits for leader to acknowledge. With all, the producer wait for leader and all replicas to ack

- Consumer must commit offset after processing using commitSync() or commitAsync()

- Start with num partitions = num consumers and increase the size accordingly. Too few can cause throughput issue while too many can cause slowness in rebalancing


- Kafka sometimes rebalances consumers ↔︎ partitions. Rebalancing involves consumers stop processing data. Use Cooperative Sticky Assignor introduced in 2.4+ to incrementally reassigning partitions.

- Log Retention should be appropriate and based off how long you want to keep your messages

- Ensure you have a database for each consumer to query for idempotency to avoid processing messages twice if you don’t use Transactional API.

- Have Dead Letter Queue and appropriate monitoring on DLQ

- Have Retry Strategy in place.

> Best Practices

- **Topic Naming**: `{environment}.{service}.{topic}`
    - Use lowercase
    - Separate by dot
    - Examples: `dev.order.created`, `prod.order.created`
- **Partitioning**: 
    - Start small (3 partitions per topic)
    - Avoid too many partitions (can impact performance and leader election)
    - Use partition keys when publishing messages to distribute messages evenly across partitions
- **Schema Management**:
    - Use Avro to define a typed-strong message and validate it before sending out
    - Use Avro to serialize and deserialize messages
    - Plan for backward/forward compatibility

- **Message Content**:
  - Include metadata (timestamp, source, version)
  - Keep messages reasonably sized (< 1MB)
  - Use compression for large messages
  - Include correlation IDs for tracking

- **Monitoring**:
  - Monitor broker metrics (CPU, memory, disk)
  - Track consumer lag which causes by consumers cannot keep up with the producer

## Notable Mentions

- **Kafka Streams**: A spin-off library that is lightweight and *in Java* 🤢 for stream processing. Think of it as ETL but with a T only 

- **Kafka Connect**: A wrapper sitting in between Kafka and external systems such as datalakes, S3, etc.

- **Kafka Streams** and **Kafka Connect** usually go hand in hand. where Connect is used to load data into Kafka and Streams is used to process data in Kafka. Check out this example from [Confluent](https://www.confluent.io/blog/hello-world-kafka-connect-kafka-streams/)